<a href="https://colab.research.google.com/github/pkp2207/Facial-Recognition-using-Facenet/blob/master/videorec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install opencv-python-headless facenet-pytorch numpy pillow
import cv2
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image, ImageFile
import numpy as np
import io
from IPython.display import display, update_display
import time
import os

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Initialize MTCNN for face detection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

# Initialize InceptionResnetV1 for face recognition
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)


In [33]:
from google.colab import files
uploaded = files.upload()


Saving WIN_20240713_15_12_12_Pro.mp4 to WIN_20240713_15_12_12_Pro.mp4


In [41]:
def recognize_faces(img):
    # Convert image to PIL format
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Detect faces
    boxes, _ = mtcnn.detect(pil_img)

    if boxes is not None:
        faces = []
        for box in boxes:
            # Crop face and convert to PIL Image
            face = pil_img.crop(box)

            # Preprocess for face recognition
            face = face.resize((160, 160), Image.LANCZOS)
            face = np.array(face)
            face = np.transpose(face, (2, 0, 1))
            face = torch.tensor(face, dtype=torch.float32)
            face = face.unsqueeze(0)

            # Calculate embeddings
            embeddings = model(face).detach().cpu().numpy()

            faces.append((box, face))

        return faces

    return []


In [44]:
def process_video(video_file, output_dir):
    cap = cv2.VideoCapture(video_file)
    frame_count = 0

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect and recognize faces
        faces = recognize_faces(frame)

        # Save recognized faces
        for i, (box, face) in enumerate(faces):
            face_np = face.squeeze(0).numpy().transpose(1, 2, 0).astype(np.uint8)
            face_img = Image.fromarray(face_np)
            face_img.save(f"{output_dir}/frame_{frame_count}_face_{i+1}.jpg")

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
    print(f"Extracted faces are saved in: {output_dir}")

# Path to the video file stored in /content
video_file = '/content/WIN_20240713_15_12_12_Pro.mp4'  # Change this to your actual video file path
output_dir = '/content/extracted_faces2'
process_video(video_file, output_dir)


Extracted faces are saved in: /content/extracted_faces2
